In [109]:
import pandas as pd
import seaborn as sns
import numpy as np
# import pykospacing
# spacer = pykospacing.Spacing()

import re
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import time
from sklearn.model_selection import train_test_split
import networkx as nx
import matplotlib.pyplot as plt


In [110]:
df = pd.read_csv('data/keyword_preprocessed_data.csv')


### 불용어 정리

In [111]:
# review 컬럼만 리스트로 저장
text=''
reviews=[]
for each_review in df['RawText']:
    reviews.append(each_review)

In [112]:
#불용어 제거
import re
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text   

review=[]
for i in reviews:  #문자가 들어있을때는 인덱스 사용하면 안됨!!!!
    a=clean_str(i)
    review.append(a)  #불용어제거한 review 저장

### 키워드 추출

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 키워드 길이 지정
n_gram_range = (1, 1)

In [114]:
# 리뷰하나씩 키워드로 분리(tfidf-vectorizer)
a= len(review) # 불용어 제거한 리뷰

token_review_list=[]

for i in range(0,a): # 리뷰하나당 처리하기위해 for문 
    review_vectorized = TfidfVectorizer(ngram_range=n_gram_range).fit([review[i]])
    token_review = review_vectorized.get_feature_names_out()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.
df['review_tfidf']=token_review_list # 토큰 단위로 나누어진 리뷰저장

### 임베딩

In [115]:
# 키워드 임베딩 모델 생성
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [116]:
# 키워드 임베딩

review_embedded_list = []
count_embedded_list = []

for i in range(0, a):
  embedding_review = model.encode([review[i]])
  review_embedded_list.append(embedding_review)

for i in df['review_tfidf']:
  embedding_count = model.encode(i)
  count_embedded_list.append(embedding_count)

### 코사인 유사도

In [117]:

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5

In [118]:
a = len(review_embedded_list)
distances = []
keywords = []

for i in range(0,a):
  distance = cosine_similarity(review_embedded_list[i], count_embedded_list[i])
  keyword = [df['review_tfidf'][i][index] for index in distance.argsort()[0][-top_n:]]

  distances.append(distance)
  keywords.append(keyword)

In [119]:
distances[0].argsort()

array([[15,  7,  1, 14, 10,  0,  2,  9,  6,  8, 13, 11,  5, 12,  3,  4]])

In [120]:
distances[0].argsort()[0][-top_n:]

array([11,  5, 12,  3,  4])

In [121]:
keywords[0]

['좋아요', '배송', '친절하시고', '만족', '만족스럽네요']

In [129]:
df['keyword'] = keywords


In [148]:
df

,ProductName,labels,RawText,review_tfidf,keyword
0,LG 울트라HD TV 75형(189cm) 75UN7850KNA+LG사운드바,1,화면이 커서 너무 좋아요 기사분도 친절하시고 배송 날짜보다 빨리 배송되어 잘 보고 ...,"[기사분도, 날짜보다, 너무, 만족, 만족스럽네요, 배송, 배송되어, 보고, 빨리,...","[좋아요, 배송, 친절하시고, 만족, 만족스럽네요]"
1,[22년 신모델] LG그램 15Z95P-GA5LK (i5-1155G7/16GB/...,1,입학 축하 선물로 노트북 뭘 살 지 이것저것 다 찾아보다가 결국 형으로 주문했습니다...,"[개봉해보니, 걱정했는데, 결국, 고급스러워졌네요, 기존, 끝내줍니다, 노트북, 도...","[포장도, 배송도, 주문했는데, 예약분부터, 주문했습니다]"
2,무선키보드 마우스 세트 저소음 옵티컬 4버튼 해킹방지 ONG ENTUS OGN-W...,1,배송은 바르고 좋은데 저소음이라고 샀는데 기대 이상이네요 그런데 마우스가 좀 빡빡한...,"[그런데, 그렇네요, 기대, 마우스가, 바르고, 배송은, 빡빡한, 샀는데, 이상이네...","[이상이네요, 배송은, 저소음이라고, 샀는데, 마우스가]"
3,(1+1 TJ 태진 블루투스 마이크 / 무선 노래방 마이크,1,코로나로 노래방을 못 가는 아이들이 어디서나 노래부를 수 있어 완전 대 박 시험 끝...,"[가는, 구성품, 기존, 끝나는, 날려버렸대요, 노래, 노래방, 노래방을, 노래부를...","[학교에서, 노래방, 마이크네요, 노래방을, 마이크는]"
4,(1+1) OO 휴대용 마스크 살균건조기,1,오늘 받았는데 어머나 너무 좋아요 마스크 바로 넣어 살균 불깜박거리고 너무 좋은데요...,"[너무, 넣어, 마스크, 바로, 받았는데, 보관하기도, 보기도, 불깜박거리고, 살균...","[마스크, 좋아요, 어머나, 좋네요, 좋은데요]"
...,...,...,...,...,...
12266,휴대용 다용도 옷걸이 건조기 제습기 HG-660DY,1,간편한 옷걸이 건조기 휴대하기도 좋고 찜찜한 옷 건조하기도 좋아 만족해요,"[간편한, 건조기, 건조하기도, 만족해요, 옷걸이, 좋고, 좋아, 찜찜한, 휴대하기도]","[만족해요, 휴대하기도, 건조기, 건조하기도, 옷걸이]"
12267,휴대폰 태블릿 거치대 자바라 130cm,1,정면 카메라 말고 카메라가 하나 더 필요해서 핸드폰 거치대를 이용해 보려고 주문했는...,"[간단하게, 거치대를, 말고, 보려고, 생각보다, 설치할, 옆으로, 이용해, 있어서...","[컴퓨터, 책상, 카메라가, 카메라, 핸드폰]"
12268,휴먼케어 감면 하이패스 시거잭전원타입 HUMC-200,0,장애인 혜택을 못 받아서 좀 아쉽지만 가성비 갑인 제품으로 추천해요 다만 지문인식 ...,"[가격, 가격까지, 가격대도, 가격도, 가격에, 가격은, 가격이, 가격이면, 가끔,...","[저렴히, 저렴한, 저렴하게, 저렴해서, 저렴해져서]"
12269,휴플러스 어깨안마기 HPR-190,1,가격이 저렴해서 구매했는데 전에 쓰던 거랑 볼의 크기가 좀 작네요 그냥 저냥 쓸 만...,"[가격이, 같아, 거랑, 구매했는데, 구입합니다, 구입해서, 그냥, 그래도, 그런대...","[구입합니다, 저렴하니, 구매했는데, 구입해서, 저렴해서]"


### 시각화

In [131]:
def extract_products_with_both_labels(df):
    # 제품별로 그룹화하고 라벨이 0과 1 모두 포함된 제품을 필터링
    filtered_df = df.groupby('ProductName').filter(lambda x: x['labels'].nunique() == 2)

    # 중복된 제품을 제거하여 유니크한 제품 목록 반환
    unique_products = filtered_df['ProductName'].unique()

    return unique_products

# 함수 호출하여 결과 출력
products_with_both_labels = extract_products_with_both_labels(df)
print(products_with_both_labels)

['(16+6형)캐리어 에어로 18단 투인원 인버터 멀티 에어컨(AMC16VX3SAGH)'
 '(1등급)(출시기념 무료설치) 위니아 인버터 창문형 에어컨(LVV06ENS)_실버'
 '(1등급)(출시기념 무료설치)위니아 인버터 창문형 에어컨(WVV06ENW)_화이트' ... '해남 꿀 고구마 베*하*카'
 '해오* 덴탈마스크 50매' '휴대용 다용도 옷걸이 건조기 제습기 HG-660DY']


In [149]:
df.loc[df['ProductName'] == '오* 무드1000 미니 가습기']

,ProductName,labels,RawText,review_tfidf,keyword
11590,오* 무드1000 미니 가습기,0,잘 사용 중이였는데 갑자기 가습 방향이 아래로만 내려가서 이상하네요 수건 깔아도 다...,"[가겠음, 가격, 가격대에, 가격도, 가격만큼, 가격에, 가격은, 가격을, 가격이,...","[중국산이라, 중국이, 저렴하지도, 중국에서, 한국에서]"
11591,오* 무드1000 미니 가습기,1,소리는 조금 나요 물은 정도 넣어야 잘 나와 요 다른 건 괜찮은 거 같아 요 이 거...,"[가가가앗가가로, 가게, 가게에, 가게에는, 가게에서, 가겠죠, 가겨, 가격, 가격...","[물때가, 물는, 물에서, 물넣고, 물분수양이]"


In [151]:
def get_keywords(product_name, label):
    # 제품명과 라벨에 해당하는 키워드 가져오기
    keywords = df[(df['ProductName'] == product_name) & (df['labels'] == label)]['keyword']
    return keywords.tolist()  # 리스트로 변환하여 반환

# 제품명과 라벨을 입력 받음
product_name = '오* 무드1000 미니 가습기'  # 여기에 원하는 제품명을 넣어주세요.
label_0 = 0
label_1 = 1

# 라벨이 0일 때 해당하는 키워드 출력
print("부정적인 리뷰에서 가장 유사한 상위 키워드 5가지:")
result_label_0 = get_keywords(product_name, label_0)
print(result_label_0)

# 라벨이 1일 때 해당하는 키워드 출력
print("\n긍정적인 리뷰에서 가장 유사한 상위 키워드 5가지:")
result_label_1 = get_keywords(product_name, label_1)
print(result_label_1)

부정적인 리뷰에서 가장 유사한 상위 키워드 5가지:
[['중국산이라', '중국이', '저렴하지도', '중국에서', '한국에서']]

긍정적인 리뷰에서 가장 유사한 상위 키워드 5가지:
[['물때가', '물는', '물에서', '물넣고', '물분수양이']]


In [152]:
df.to_csv('data/keyword_extractor_data.csv', index=False) # 혹시 몰라서 저장